In [1]:
!pip install streamlit pyngrok
!pip install gradio



import numpy as np
import pandas as pd
import nltk
import re

nltk.download('punkt')

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [20]:
anime = pd.read_csv("anime-dataset.csv")

In [21]:
anime.shape
anime.head()

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43.0,78525.0,914193.0,1771505.0,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602.0,1448.0,206248.0,360978.0,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246.0,15035.0,356739.0,727252.0,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795.0,613.0,42829.0,111931.0,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126.0,14.0,6413.0,15001.0,https://cdn.myanimelist.net/images/anime/7/215...


In [22]:
anime['Genres'].value_counts()

,count
Genres,
UNKNOWN,5145
Comedy,2299
Fantasy,1358
Hentai,1193
Avant Garde,646
...,...
"Adventure, Award Winning, Mystery, Romance, Sci-Fi",1
"Action, Boys Love, Supernatural",1
"Action, Avant Garde, Horror, Sci-Fi",1


In [23]:
anime.dropna(inplace=True)

In [24]:
anime = anime[~anime['Genres'].str.contains('Boys Love|Hentai|Erotica|GirlsLove|UNKNOWN',regex = True)]
anime.reset_index(drop=True,inplace=True)
anime.shape

(18553, 24)

In [25]:
anime = anime[['anime_id','Name','Genres','Synopsis','Image URL']]

In [26]:
anime['Genres']=anime['Genres'].apply(lambda x : x.split(","))
anime['Genres']=anime['Genres'].apply(lambda x:[i.replace(" ","") for i in x])
anime['Genres']


,Genres
0,"[Action, AwardWinning, Sci-Fi]"
1,"[Action, Sci-Fi]"
2,"[Action, Adventure, Sci-Fi]"
3,"[Action, Drama, Mystery, Supernatural]"
4,"[Adventure, Fantasy, Supernatural]"
...,...
18548,[AvantGarde]
18549,[AvantGarde]
18550,"[Comedy, Fantasy, SliceofLife]"
18551,"[Action, Adventure, Fantasy]"


In [27]:
anime['tags'] = anime['Genres'].astype(str) + " " + anime['Synopsis'].astype(str)
anime['tags'][0]


"['Action', 'AwardWinning', 'Sci-Fi'] Crime is timeless. By the year 2071, humanity has expanded across the galaxy, filling the surface of other planets with settlements like those on Earth. These new societies are plagued by murder, drug use, and theft, and intergalactic outlaws are hunted by a growing number of tough bounty hunters.\n\nSpike Spiegel and Jet Black pursue criminals throughout space to make a humble living. Beneath his goofy and aloof demeanor, Spike is haunted by the weight of his violent past. Meanwhile, Jet manages his own troubled memories while taking care of Spike and the Bebop, their ship. The duo is joined by the beautiful con artist Faye Valentine, odd child Edward Wong Hau Pepelu Tivrusky IV, and Ein, a bioengineered Welsh Corgi.\n\nWhile developing bonds and working to catch a colorful cast of criminals, the Bebop crew's lives are disrupted by a menace from Spike's past. As a rival's maniacal plot continues to unravel, Spike must choose between life with his 

In [28]:
anime = anime[['anime_id', 'Name', 'tags', 'Image URL','Synopsis']]

In [29]:
anime['tags'] = anime['tags'].apply(lambda x: x.lower())
anime['tags']

<ipython-input-29-a760c01c894c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime['tags'] = anime['tags'].apply(lambda x: x.lower())


,tags
0,"['action', 'awardwinning', 'sci-fi'] crime is ..."
1,"['action', 'sci-fi'] another day, another boun..."
2,"['action', 'adventure', 'sci-fi'] vash the sta..."
3,"['action', 'drama', 'mystery', 'supernatural']..."
4,"['adventure', 'fantasy', 'supernatural'] it is..."
...,...
18548,['avantgarde'] music video for the song thaila...
18549,['avantgarde'] music video for the song energy...
18550,"['comedy', 'fantasy', 'sliceoflife'] no descri..."
18551,"['action', 'adventure', 'fantasy'] no descript..."


In [30]:
def clean_text(text):
   return re.sub(r'[^\w\s]', '', text)

# Apply the cleaning function to the 'tags' column
anime['tags'] = anime['tags'].apply(clean_text)

# Print the cleaned 'tags' column
print(anime['tags'])

0        action awardwinning scifi crime is timeless by...
1        action scifi another day another bountysuch is...
2        action adventure scifi vash the stampede is th...
3        action drama mystery supernatural robin sena i...
4        adventure fantasy supernatural it is the dark ...
                               ...                        
18548    avantgarde music video for the song thailand b...
18549    avantgarde music video for the song energy by ...
18550    comedy fantasy sliceoflife no description avai...
18551    action adventure fantasy no description availa...
18552    action adventure fantasy scifi no description ...
Name: tags, Length: 18553, dtype: object


In [32]:
ps = PorterStemmer()

def stem(text):
    words = word_tokenize(text)
    stemmed_words = [ps.stem(word) for word in words]
    return " ".join(stemmed_words)



In [33]:
anime['tags']=anime['tags'].apply(stem)
anime['tags']

,tags
0,action awardwin scifi crime is timeless by the...
1,action scifi anoth day anoth bountysuch is the...
2,action adventur scifi vash the stamped is the ...
3,action drama mysteri supernatur robin sena is ...
4,adventur fantasi supernatur it is the dark cen...
...,...
18548,avantgard music video for the song thailand by...
18549,avantgard music video for the song energi by s...
18550,comedi fantasi sliceoflif no descript avail fo...
18551,action adventur fantasi no descript avail for ...


In [34]:
# Print the tags directly
tags = anime[anime['Name'] == 'Viewtiful Joe']
display(tags)


,anime_id,Name,tags,Image URL,Synopsis
244,278,Viewtiful Joe,action comedi scifi joe a redhead movi buff an...,https://cdn.myanimelist.net/images/anime/1/278...,"Joe, a red-headed movie buff, and Silvia, his ..."


In [35]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000,stop_words="english")

cv_vectors = cv.fit_transform(anime['tags']).toarray()

In [45]:
similarity_cv = cosine_similarity(cv_vectors, cv_vectors)


In [46]:
similarity_cv.shape

(18553, 18553)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 5.0 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import pandas as pd
import numpy as np

def calculate_similarity(anime_name):
    # Find the index of the given anime name
    anime_index = anime[anime['Name'] == anime_name].index[0]

    # Get the similarity scores for the selected anime
    similarity_scores = similarity_cv[anime_index]

    # Get the top N indices of similar animes
    top_n = 5
    top_n_indices = np.argsort(-similarity_scores)[:10]
    print('TOP',top_n_indices)

    # Create a list to store tuples of indices and values
    similarity_info = [(i, similarity_scores[i]) for i in top_n_indices]
    print('SIMI INFO',similarity_info)

    return similarity_info

def recommend(anime_name):
    # Get similarity info for the chosen anime
    similarity_info = calculate_similarity(anime_name)

    recommended_animes = []
    recommended_animes_posters = []
    recommended_animes_synopsises = []

    # Collect names and posters for the top recommendations
    for index, score in similarity_info:
        recommended_animes.append(anime.iloc[index]['Name'])
        recommended_animes_posters.append(anime.iloc[index]['Image URL'])
        recommended_animes_synopsises.append(anime.iloc[index]['Synopsis'])

    return recommended_animes, recommended_animes_posters, recommended_animes_synopsises



def recommend_anime(anime_name):
    names, posters ,synopsises= recommend(anime_name)
    recommendations = []
    for name, poster , synopsis in zip(names, posters,synopsises):
        recommendations.append((name, poster,synopsis))
    return recommendations


dropdown = gr.Dropdown(choices=anime['Name'].values.tolist(), label="Choose Anime")
output = gr.HTML()

def format_output(anime_name):
    recommendations = recommend_anime(anime_name)
    result = "<div>"

    for name, poster, synopsis in recommendations:
        result += '<div style="margin-bottom: 20px;">'  # Add space between tiles
        result += '<div style="display: flex; align-items: center;">'
        result += f'<img src="{poster}" style="width:150px;height:200px; margin-right: 20px;">'
        result += f'<div style="padding: 10px;"><p><strong>{name}</strong></p>'
        result += f'<p>{synopsis}</p></div>'
        result += '</div></div>'  # Close the inner div

    result += "</div>"
    return result

interface = gr.Interface(
    fn=format_output,
    inputs=dropdown,
    outputs=output,
    title="Anime Recommender",
    allow_flagging='never',
    fill_width = True
)

if __name__ == "__main__":
    interface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://04a6cac206ecc2bddf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


11
TOP [   11 10871  3013  3386  1075  6694 11748  5653  1076  7302]
SIMI INFO [(11, 1.0), (10871, 0.495491027164164), (3013, 0.415901532503537), (3386, 0.4137638166982224), (1075, 0.39679187388094006), (6694, 0.3751272212872719), (11748, 0.3712473257527238), (5653, 0.3550924212632751), (1076, 0.33808994784557644), (7302, 0.33545785027252367)]
10
TOP [   10 18427 18187 10186  1474  8152  3099   539  5794  5193]
SIMI INFO [(10, 1.0), (18427, 0.9952718411247496), (18187, 0.9952718411247496), (10186, 0.42896059000888737), (1474, 0.4128197767394713), (8152, 0.40962209678224715), (3099, 0.3521515739748789), (539, 0.3474540920416088), (5794, 0.33290015143059165), (5193, 0.31386127271279984)]
477
TOP [ 477  969 1331 9957 6826  970 4549 2945 4677 4453]
SIMI INFO [(477, 0.9999999999999997), (969, 0.5900260138554375), (1331, 0.5632196212257321), (9957, 0.5504435010141958), (6826, 0.5172821661655906), (970, 0.4833631227672434), (4549, 0.47875966445517437), (2945, 0.47237533163385365), (4677, 0.46